In [7]:
# Suppress debugging information about GPU
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import os.path
import sys
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from numpy import mean, std
from keras.initializers import RandomUniform
from keras.layers import Dense, InputLayer
from keras.losses import MeanSquaredError
from keras.metrics import BinaryAccuracy
from keras.models import Sequential
from keras.optimizers import SGD
from keras.regularizers import L2, L1
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from pathlib import Path

In [8]:
def read_ds(path):
  """
  parse CSV data set and
  returns a tuple (input, target)
  """
  df = pd.read_csv(path, sep=" ", names=['NaN','y','x1','x2','x3','x4','x5','x6','garbage'])
  y, df = df['y'], df.drop(columns=['NaN','garbage','y'])
  
  # One-hot encoding categorical variables
  df = pd.get_dummies(df, columns=['x1','x2','x3','x4','x5','x6']).astype('int')

  return (df, y)

In [9]:
# Datasets Path
TR_PATH = "./monks/datasets/monks-1.train"
TS_PATH = "./monks/datasets/monks-1.test"

X_train, y_train = read_ds(TR_PATH)
X_test, y_test = read_ds(TS_PATH)

In [10]:
# Function that builds up a NN
def get_NN(meta, neurons=4, lr=0.02, alpha=0.5, hidden_activation="tanh", initialization_seed=42):

  # random initialization
  random_init = RandomUniform(seed=initialization_seed)

  # 1 hidden layer
  NN_model = Sequential()
  NN_model.add(InputLayer(input_shape=(meta["n_features_in_"],)))
  NN_model.add(Dense(units=neurons,  activation=hidden_activation, kernel_initializer=random_init, 
                bias_initializer=random_init, kernel_regularizer=None)
              )
  NN_model.add(Dense(units=1, activation="sigmoid", kernel_initializer=random_init, 
                bias_initializer=random_init, kernel_regularizer=None)
              )

  NN_model.compile(optimizer=SGD(learning_rate=lr, momentum=alpha),loss=MeanSquaredError(), metrics=BinaryAccuracy())

  return NN_model

In [11]:
#utility to define a list of grids for GridSearchCV (as some values can conflict with other ones)
def list_grids(grids_dict):
    return [grids_dict[item] for item in grids_dict]

#utility to print results of model selection phase
def print_results(model_name,best_score,std_of_best, best_params):
    metric="mse"
    print(f"mean validation {metric} : {best_score} +/- {std_of_best} (std on val)")
    print("best parameters from grid search", best_params)

# Function that prints the classification report
def print_report_score(test_label, test_pred):
    print(classification_report(test_label, test_pred, target_names=['0', '1']))

In [12]:
NN_MONK1_GRID_DICT = {
  "unique_entry":{
    #"model__X_len": [len(X_train)],
    "model__lr":[0.1,0.2,0.3,0.4],    # [0.255]
    "model__alpha": [0.5,0.6,0.7,0.8],# [0.625]
    "model__hidden_activation": ["tanh"],
    "model__neurons": [2,3,4,5], # [5]
    "batch_size":[4,8,16,32], # 4
    "model__initialization_seed":list(range(50,55))
  }
}

NN = KerasClassifier(
        model=get_NN,
        loss="mse",
        optimizer="sgd",
        shuffle=True,
        verbose=False,
        epochs=600,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="loss", min_delta=1e-4, patience=15, restore_best_weights=True)]
    )

custom_scores_monk = {
    "accuracy": "accuracy",
    "mse": make_scorer(mean_squared_error,greater_is_better=False)
}

grid = GridSearchCV(NN,
                    param_grid=list_grids(NN_MONK1_GRID_DICT),
                    scoring=custom_scores_monk,
                    refit="mse",
                    cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                    return_train_score=True,
                    n_jobs=-1,
        )
grid.fit(X_train, y_train)
print("Best parameters: " + str(grid.best_params_) + " score: " + str(grid.best_score_))

/home/marco/Programmi/PyEnvs/ML/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 